In [1]:
# Importing required libraries
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import performance_metrics, cross_validation
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

In [2]:
data = pd.read_csv("../data/merged_final_1h.csv")

In [3]:
data

,dt_start_utc,windspeed_ms,epex_da_de_eur_mwh,solar_mw_fiftyhertz,solar_mw_tennet,solar_mw_amprion,solar_mw_transnetbw,solar_mw_nrv,fc_da,fc_load_50H,...,offshore_mw_nrv,onshore_mw_fiftyhertz,onshore_mw_tennet,onshore_mw_amprion,onshore_mw_transnetbw,onshore_mw_nrv,month,weekday,hour,imbalance_price_target
0,2019-01-01 00:00:00,13.7925,10.07,0.0,0.00,0.0,0.0,0.00,29.730,6150.5,...,5684.34,9231.31,11624.89,3039.50,105.45,24001.15,1,1,0,-3.465
1,2019-01-01 01:00:00,14.3600,-4.08,0.0,0.00,0.0,0.0,0.00,26.050,6029.0,...,5334.19,10012.16,12762.32,3576.50,112.04,26463.02,1,1,1,-3.735
2,2019-01-01 02:00:00,15.0180,-9.91,0.0,0.00,0.0,0.0,0.00,23.760,6057.0,...,5278.54,10852.08,13988.91,4094.75,164.29,29100.03,1,1,2,-9.250
3,2019-01-01 03:00:00,14.8220,-7.41,0.0,0.00,0.0,0.0,0.00,20.950,6122.5,...,5160.90,11761.89,14809.73,4685.25,226.52,31483.39,1,1,3,11.355
4,2019-01-01 04:00:00,14.5930,-12.55,0.0,0.00,0.0,0.0,0.00,15.770,6152.0,...,5001.96,13125.93,15131.73,5024.50,315.82,33597.98,1,1,4,-4.925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15331,2020-09-30 19:00:00,6.1635,49.92,0.0,0.03,0.0,0.0,0.03,53.555,11108.5,...,4756.97,651.60,3648.29,1620.00,179.00,6098.89,9,2,19,16.450
15332,2020-09-30 20:00:00,5.9385,42.79,0.0,0.05,0.0,0.0,0.05,46.200,10454.5,...,5196.46,707.77,3953.82,1977.75,259.00,6898.34,9,2,20,26.050
15333,2020-09-30 21:00:00,6.0630,35.02,0.0,0.05,0.0,0.0,0.05,40.030,9635.5,...,5339.31,779.20,3957.25,2123.75,284.25,7144.45,9,2,21,56.265
15334,2020-09-30 22:00:00,6.2825,34.40,0.0,0.06,0.0,0.0,0.06,37.970,9068.0,...,5396.54,847.60,4031.58,2152.50,326.25,7357.93,9,2,22,41.335


In [4]:
# set timestamp as index
data['dt_start_utc'] = pd.to_datetime(data['dt_start_utc'])

In [5]:
df = data[['dt_start_utc', 'imbalance_price_target', 'fc_da', 'fc_load_DE', 'windspeed_ms', 'fc_onshore_DE', 'fc_offshore_DE', 'fc_solar_DE', 'weekday', 'hour', 'month']]

In [6]:
df.rename(columns={'dt_start_utc': 'ds', 'imbalance_price_target': 'y'}, inplace=True)

In [7]:
df

,ds,y,fc_da,fc_load_DE,windspeed_ms,fc_onshore_DE,fc_offshore_DE,fc_solar_DE,weekday,hour,month
0,2019-01-01 00:00:00,-3.465,29.730,41218.5,13.7925,20618.5,5021.0,0.0,1,0,1
1,2019-01-01 01:00:00,-3.735,26.050,40139.0,14.3600,22352.0,5028.0,0.0,1,1,1
2,2019-01-01 02:00:00,-9.250,23.760,39917.0,15.0180,24035.0,4978.0,0.0,1,2,1
3,2019-01-01 03:00:00,11.355,20.950,40282.0,14.8220,25475.0,4908.0,0.0,1,3,1
4,2019-01-01 04:00:00,-4.925,15.770,40528.0,14.5930,26538.5,4884.0,0.0,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...
15331,2020-09-30 19:00:00,16.450,53.555,56709.0,6.1635,6282.0,3545.5,0.0,2,19,9
15332,2020-09-30 20:00:00,26.050,46.200,53241.0,5.9385,7379.0,3931.0,0.0,2,20,9
15333,2020-09-30 21:00:00,56.265,40.030,49149.0,6.0630,8190.0,4156.5,0.0,2,21,9
15334,2020-09-30 22:00:00,41.335,37.970,46540.0,6.2825,7684.5,4768.0,0.0,2,22,9


### Modeling

In [8]:
def train_test_split(df):
    n = int(len(df)*0.7)
    train, test = df.iloc[:n], df.iloc[n:]
    return train,test

In [9]:
train, test = train_test_split(df)

In [10]:
m = Prophet()
m.fit(train)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Initial log joint probability = -80.7587


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       23394.5    0.00469751       540.884           1           1      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       23399.8     0.0010439       158.242           1           1      252   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       23401.8   0.000360909       167.136      0.1318      0.1318      372   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     319       23402.3    8.8458e-05           286   1.819e-07       0.001      438  LS failed, Hessian reset 
     399       23403.2   0.000192169       78.8162           1           1      537   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       23403.6     8.024e-05       66.4058           1           1      6

In [11]:
future = m.make_future_dataframe(periods=24, freq='H')

In [12]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
10754,2020-03-24 02:00:00,14.905135,-35.540123,63.456632
10755,2020-03-24 03:00:00,17.108067,-34.335517,69.924998
10756,2020-03-24 04:00:00,21.009152,-28.384272,71.659301
10757,2020-03-24 05:00:00,25.048038,-26.616417,76.585995
10758,2020-03-24 06:00:00,27.599341,-23.134962,78.463988


In [13]:
# forecasts plot
plot_plotly(m, forecast)

In [14]:
# components
plot_components_plotly(m, forecast)

In [17]:
#for i in range(len(test_2019)):
forecasts_1h = []
test_imb = test
train_imb = train
print(len(test_imb))
for i in range(len(test_imb)):
    train_imb = pd.concat([train_imb, pd.DataFrame(test_imb.iloc[i]).T])
    m3 = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
    m3.add_seasonality(name='half day', period=0.5, fourier_order=5)
    m3.add_country_holidays(country_name='DE')
    m3.add_regressor('fc_da')
    m3.add_regressor('fc_load_DE')
    m3.add_regressor('windspeed_ms')
    m3.add_regressor('fc_onshore_DE')
    m3.add_regressor('fc_offshore_DE')
    m3.fit(train_imb)
    future3 = m3.make_future_dataframe(periods=1, freq='H')
    future3['fc_da'] = df[df['ds'] <= future3.iloc[-1]['ds']]['fc_da']
    future3['fc_load_DE'] = df[df['ds'] <= future3.iloc[-1]['ds']]['fc_load_DE']
    future3['windspeed_ms'] = df[df['ds'] <= future3.iloc[-1]['ds']]['windspeed_ms']
    future3['fc_onshore_DE'] = df[df['ds'] <= future3.iloc[-1]['ds']]['fc_onshore_DE']
    future3['fc_offshore_DE'] = df[df['ds'] <= future3.iloc[-1]['ds']]['fc_offshore_DE']
    forecast3 = m3.predict(future3)
    forecast_1h = forecast['yhat'][0]
    print(i)
    forecasts_1h.append(forecast_1h)

4601
Initial log joint probability = -37.5173
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       23938.1    0.00054241       245.391           1           1      112   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     176       23938.8   7.10765e-05       192.231   2.499e-07       0.001      241  LS failed, Hessian reset 
     199       23938.9   0.000340523       139.691           1           1      268   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     277       23938.9   5.56633e-06       44.0934      0.3145           1      370   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
0
Initial log joint probability = -948.845
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       23881.4    0.00132693       274.973      0.4503   